In [17]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time

In [18]:
url = "https://6gkb.by/uslugi/prejskuranty-dlya-grazhdan-respubliki-belarus"

In [19]:
def get_soup(url):
    
    # Прописываем нестандартный заголовок, чтобы сайт не принял нас за бота, который парсит его данные :)
    headers = requests.utils.default_headers()
    headers.update({
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0',
    })
    
    response = requests.get(url=url, headers=headers)

    if not response.ok:
        raise ValueError("no content to parse")

    return BeautifulSoup(response.content.decode(), "lxml")

In [20]:
#забираем ссылки на подразделы анализов hrefs
source = "https://www.6gkb.by"
URL = "https://6gkb.by/uslugi/prejskuranty-dlya-grazhdan-respubliki-belarus"
hrefs = []
soup = get_soup(url=URL)

tags = soup.find(name="div", class_ = "uk-grid uk-grid-small uk-grid-match")

for tag in tags.find_all(name="div", class_ = "uk-width-small-2-3 uk-width-medium-3-4"):

# tag = soup.find(name="div", class_ = "uk-width-small-2-3 uk-width-medium-3-4")
    #print(tag)
    
    for a in tag.find_all("a"):

        url_capter = a.get("href")
        name_capter = a.text
        hrefs = np.append(hrefs, url_capter)
        #print(f"{url_capter = }, {name_capter = }")
    
print(hrefs[19])

/uslugi/prejskuranty-dlya-grazhdan-respubliki-belarus/laboratornye-metody-issledovaniya/laboratornaya-diagnostika


In [21]:
# парсим сами анализы
source = "https://www.6gkb.by"
np_analisys= np.array([])
url_chapter = hrefs[19]
URL = source + url_chapter
soup = get_soup(URL)

n_td = 6
analisys_comment = ""


for  tag in soup.find_all("table"):
    chapter = tag.find_all("tr")[3].text
    
        
    for row in tag.find_all("tr"):        


        if n_td == len(row.find_all("td")):
            t_columns = row.find_all("td")
            analisys_name = t_columns[1].text
            analisys_cost = t_columns[5].text
            
            np_analisys = np.append(np_analisys, (source, chapter, analisys_name, analisys_cost, analisys_comment))

    time.sleep(3) # ставим задержку 3 секунды после парсинга каждого раздела
print(np_analisys.reshape(-1, 5))

[['https://www.6gkb.by' '\n\xa0\n\nЛабораторная диагностика\n\n'
  '\nОпределение осмотической резистентности эритроцитов фотометрическим методом\n'
  '\n19.70\n' '']
 ['https://www.6gkb.by' '\n\xa0\n\nЛабораторная диагностика\n\n'
  '\nГемостазиограмма (пункты 1.2. + 1.4.1. + 1.5. + 6.40.1. + 6.40.2. + 6.40.3. + 6.6.4.)*\n'
  '\n33.41\n' '']
 ['https://www.6gkb.by' '\n\xa0\n\nЛабораторная диагностика\n\n'
  '\nКопрограмма (пункты 1.2. + 2.8.1. + 2.8.2. + 2.8.3.)\n' '\n11.91\n'
  '']
 ...
 ['https://www.6gkb.by' '\n\xa0\n\nЛабораторная диагностика\n\n'
  '\nОпределение иммуноглобулинов (антител) одного класса к Хламидии Трохоматис ( Ig А ) с ручным расчетом коэффициента позитивности и титра антител*\n'
  '\n19.45\n' '']
 ['https://www.6gkb.by' '\n\xa0\n\nЛабораторная диагностика\n\n'
  '\nОпределение иммуноглобулинов (антител) одного класса к хламидии трохоматис-IgМ (с абсорбентом РФ)*\n'
  '\n22.89\n' '']
 ['https://www.6gkb.by' '\n\xa0\n\nЛабораторная диагностика\n\n'
  '\nНагрузочны

In [22]:
df_6gkb = pd.DataFrame(np_analisys.reshape(-1, 5), columns=["source", "chapter", "analisys_name", "analisys_cost", "analisys_comment"])
df_6gkb

,source,chapter,analisys_name,analisys_cost,analisys_comment
0,https://www.6gkb.by,\n \n\nЛабораторная диагностика\n\n,\nОпределение осмотической резистентности эрит...,\n19.70\n,
1,https://www.6gkb.by,\n \n\nЛабораторная диагностика\n\n,\nГемостазиограмма (пункты 1.2. + 1.4.1. + 1.5...,\n33.41\n,
2,https://www.6gkb.by,\n \n\nЛабораторная диагностика\n\n,\nКопрограмма (пункты 1.2. + 2.8.1. + 2.8.2. +...,\n11.91\n,
3,https://www.6gkb.by,\n \n\nЛабораторная диагностика\n\n,\nОбщий анализ крови с тромбоцитами (пункты 1....,\n15.12\n,
4,https://www.6gkb.by,\n \n\nЛабораторная диагностика\n\n,\nОбщий анализ крови (пункты 1.2.+ 1.4..+ 3.7....,\n11.78\n,
...,...,...,...,...,...
270,https://www.6gkb.by,\n \n\nЛабораторная диагностика\n\n,"\nОпределение антител (М, G ) к SARS-Cov-2 в к...",\n15.15\n,
271,https://www.6gkb.by,\n \n\nЛабораторная диагностика\n\n,\nОпределение иммуноглобулинов (антител) одног...,\n19.51\n,
272,https://www.6gkb.by,\n \n\nЛабораторная диагностика\n\n,\nОпределение иммуноглобулинов (антител) одног...,\n19.45\n,
273,https://www.6gkb.by,\n \n\nЛабораторная диагностика\n\n,\nОпределение иммуноглобулинов (антител) одног...,\n22.89\n,


In [3]:
import datetime as dt
from typing import List, Optional
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
from abc import ABC, abstractmethod
from WebDriver import WebDriver, Analysis

In [4]:
class Analysis:
    
    def __init__(self, source: str, chapter: str|None, analysis_name: str, analysis_cost:str, analysis_comment: str,):              

        self.source    = source
        self.chapter     = chapter
        self.analysis_name   = analysis_name
        self.analysis_cost   = analysis_cost
        self.analysis_comment  = analysis_comment

    def __repr__(self) -> str:
        return f"source = {self.source}, chapter = {self.chapter}, analysis_name = {self.analysis_name}, analysis_cost = {self.analysis_cost}, analysis_comment = {self.analysis_comment}"
    
    def to_dict(self) -> dict:
        return self.__dict__


In [5]:
class WebDriver(ABC):

    def __init__(self):
        self.urls: List[str] = []
            
    def _scan_page(self, url:str):
        # Прописываем нестандартный заголовок, чтобы сайт не принял нас за бота, который парсит его данные :)
        headers = requests.utils.default_headers()
        headers.update({
            'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0',
            })

        response = requests.get(url=url, headers=headers)

        if not response.ok:
            raise ValueError("no content to parse")

        return BeautifulSoup(response.content.decode(), "lxml")            

    @abstractmethod
    def get_analysis(self) :  
        ...


In [13]:
class Driver6gkb(WebDriver):
    URL = "https://www.6gkb.by" 
    NAME = "6gkb.by"

    def __init__(self):
        super().__init__()
        self._chapters: List[str] = []

    def scan(self):

        #забираем ссылки на подразделы анализов hrefs
        url = "https://6gkb.by/uslugi/prejskuranty-dlya-grazhdan-respubliki-belarus"
        
        soup = self._scan_page(url=url)

        tags = soup.find(name="div", class_ = "uk-grid uk-grid-small uk-grid-match")

        for tag in tags.find_all(name="div", class_ = "uk-width-small-2-3 uk-width-medium-3-4"):

            for a in tag.find_all("a"):

                url_capter = Driver6gkb.URL + a.get("href")
                name_capter = a.get_text(strip=True)
                self.urls.append(url_capter)
                self._chapters.append(name_capter)
    
        return (self)

    def get_analysis(self):
        i = 0
        #         analysis_list = []
        analysis_dict = {}
        url = self.urls[19]
        soup = self._scan_page(url)
        n_td = 6
            
        for tag in soup.find_all("table"):
            chapter = tag.find_all("tr")[3].text
    
            for row in tag.find_all("tr"):        

                if n_td == len(row.find_all("td")):
                    t_columns = row.find_all("td")
                    analysis_name = t_columns[1].text.strip()
                    analysis_cost = t_columns[5].text.strip()
                    analysis_comment = ""

                    analysis = Analysis(Driver6gkb.NAME, chapter, analysis_name, analysis_cost, analysis_comment)
                    analysis_dict[i] = analysis.to_dict()
 
                    i += 1
            time.sleep(3) # ставим задержку 3 секунды после парсинга каждого раздела  
            
        return analysis_dict

In [14]:
d6kb = Driver6gkb()

In [15]:
d6kb.scan()

In [16]:
d6kb.get_analysis()

{0: {'source': '6gkb.by',
  'chapter': '\n\xa0\n\nЛабораторная диагностика\n\n',
  'analysis_name': 'Определение осмотической резистентности эритроцитов фотометрическим методом',
  'analysis_cost': '19.70',
  'analysis_comment': ''},
 1: {'source': '6gkb.by',
  'chapter': '\n\xa0\n\nЛабораторная диагностика\n\n',
  'analysis_name': 'Гемостазиограмма (пункты 1.2. + 1.4.1. + 1.5. + 6.40.1. + 6.40.2. + 6.40.3. + 6.6.4.)*',
  'analysis_cost': '33.41',
  'analysis_comment': ''},
 2: {'source': '6gkb.by',
  'chapter': '\n\xa0\n\nЛабораторная диагностика\n\n',
  'analysis_name': 'Копрограмма (пункты 1.2. + 2.8.1. + 2.8.2. + 2.8.3.)',
  'analysis_cost': '11.91',
  'analysis_comment': ''},
 3: {'source': '6gkb.by',
  'chapter': '\n\xa0\n\nЛабораторная диагностика\n\n',
  'analysis_name': 'Общий анализ крови с тромбоцитами (пункты 1.2.+ 1.4.+ 3.7.+ 3.9.1.+ 3.26.3. + 3.6.1.)*',
  'analysis_cost': '15.12',
  'analysis_comment': ''},
 4: {'source': '6gkb.by',
  'chapter': '\n\xa0\n\nЛабораторная ди